# Exercise3：Pokemon classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/taipeitechmmslab/MMSLAB-TF2/blob/master/Exercise/Exercise3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/taipeitechmmslab/MMSLAB-TF2/blob/master/Exercise/Exercise3.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

https://www.kaggle.com/terminus7/pokemon-challenge

- Name：寶可夢名子
- Type 1：寶可夢的屬性
- Type 2：寶可夢的屬性(寶可夢可能擁有兩種屬性)
- HP：生命直
- Attack：攻擊力
- Defense：守備力
- Sp. Atk：特殊攻擊力
- Sp. Def：特殊防禦力
- Speed：速度
- Generation：進化階段
- Legendary：神獸 (1=神獸, 0=一般)

### Import

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.utils import plot_model
from IPython.display import Image

### 讀取數據並分析

讀取800隻寶可夢數據：

In [3]:
url='https://raw.githubusercontent.com/rainy0215/NTUT_MS/master/Lab3/pokemon.csv'
pokemon_df = pd.read_csv(url)  # 讀取寶可夢數據
pokemon_df.head()  # 顯示頭5比資料

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


將"#"數據設定為索引值：

In [4]:
pokemon_df= pokemon_df.set_index("#")  # 將索引設定為'#'列
pokemon_df.head()  # 顯示頭5比資料

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


讀取寶可夢對戰數據：

In [5]:
url='https://raw.githubusercontent.com/rainy0215/NTUT_MS/master/Lab3/combats.csv'
combats_df = pd.read_csv(url)  # 讀取對戰數據
combats_df.head()  # 顯示頭5比資料

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


### 補齊缺失 資料

檢查寶可夢數據是否有缺失資料：

In [6]:
pokemon_df.info()  # 顯示精靈寶可夢的數據資訊

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 1 to 800
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        799 non-null    object
 1   Type 1      800 non-null    object
 2   Type 2      414 non-null    object
 3   HP          800 non-null    int64 
 4   Attack      800 non-null    int64 
 5   Defense     800 non-null    int64 
 6   Sp. Atk     800 non-null    int64 
 7   Sp. Def     800 non-null    int64 
 8   Speed       800 non-null    int64 
 9   Generation  800 non-null    int64 
 10  Legendary   800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 69.5+ KB


查看Type2每個類別的數量：

透過傳入參數dropna=False，可以將缺失數據（NaN）也考慮進去，NaN代表寶可夢並沒有第二種屬性。

In [7]:
pokemon_df["Type 2"].value_counts(dropna =False)

NaN         386
Flying       97
Ground       35
Poison       34
Psychic      33
Fighting     26
Grass        25
Fairy        23
Steel        22
Dark         20
Dragon       18
Ghost        14
Water        14
Ice          14
Rock         14
Fire         12
Electric      6
Normal        4
Bug           3
Name: Type 2, dtype: int64

填補缺失數據：使用empty將缺失的欄位填上。

In [8]:
pokemon_df["Type 2"].fillna('empty',inplace=True)
pokemon_df["Type 2"].value_counts()

empty       386
Flying       97
Ground       35
Poison       34
Psychic      33
Fighting     26
Grass        25
Fairy        23
Steel        22
Dark         20
Dragon       18
Rock         14
Ghost        14
Water        14
Ice          14
Fire         12
Electric      6
Normal        4
Bug           3
Name: Type 2, dtype: int64

### 數據前處理
檢查資料型態

In [9]:
print(combats_df.dtypes)  # 顯示對戰數據的資料型態
print('-' * 30)
print(pokemon_df.dtypes)  # 顯示寶可夢數據的資料型態

First_pokemon     int64
Second_pokemon    int64
Winner            int64
dtype: object
------------------------------
Name          object
Type 1        object
Type 2        object
HP             int64
Attack         int64
Defense        int64
Sp. Atk        int64
Sp. Def        int64
Speed          int64
Generation     int64
Legendary       bool
dtype: object


轉換數據格式

In [10]:
pokemon_df['Type 1'] = pokemon_df['Type 1'].astype('category')  # 將'Type 1'轉成category型態
pokemon_df['Type 2'] = pokemon_df['Type 2'].astype('category')  # 將'Type 2'轉成category型態
pokemon_df['Legendary'] = pokemon_df['Legendary'].astype('int')  # 將'Legendary'轉成int型態
pokemon_df.dtypes  # 顯示目前寶可用數據集資料型態

Name            object
Type 1        category
Type 2        category
HP               int64
Attack           int64
Defense          int64
Sp. Atk          int64
Sp. Def          int64
Speed            int64
Generation       int64
Legendary        int64
dtype: object

將寶可夢的Type1和Type2轉為One-hot Encoding表示：

In [11]:
df_type1_one_hot = pd.get_dummies(pokemon_df['Type 1'])  # 取得'Type 1'數據的One-hot編碼
df_type1_one_hot.head()  # 顯示頭5比資料

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


使用Pandas的get_dummies函數，取得Type2（寶可夢第二種屬性）的One-hot Encoding：

In [12]:
df_type2_one_hot = pd.get_dummies(pokemon_df['Type 2'])  # 取得'Type 2'數據的One-hot編碼
df_type2_one_hot.head()  # 顯示頭5比資料

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


將兩組One-hot Encoding合併回數據集：

In [13]:
# 將上方兩個One-hot編碼資料加起來，將缺少的值補0，並轉呈int64型態
combine_df_one_hot = df_type1_one_hot.add(df_type2_one_hot, fill_value=0).astype('int64')
# 將顯示列數設定為30，不然會有部份資料無法顯示
pd.options.display.max_columns = 30
# 將One-hot編碼資料加到寶可夢數據中
pokemon_df = pokemon_df.join(combine_df_one_hot)
pokemon_df.head()  # 顯示頭5比資料

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,Charmander,Fire,empty,39,52,43,60,50,65,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


將寶可夢屬性轉為數值表示（0, 1, 2, …18）：透過cat.categories查詢類別的標籤。

In [14]:
dict(enumerate(pokemon_df['Type 2'].cat.categories))

{0: 'Bug',
 1: 'Dark',
 2: 'Dragon',
 3: 'Electric',
 4: 'Fairy',
 5: 'Fighting',
 6: 'Fire',
 7: 'Flying',
 8: 'Ghost',
 9: 'Grass',
 10: 'Ground',
 11: 'Ice',
 12: 'Normal',
 13: 'Poison',
 14: 'Psychic',
 15: 'Rock',
 16: 'Steel',
 17: 'Water',
 18: 'empty'}

透過cat.codes可以取得類別的編碼值。

In [15]:
pokemon_df['Type 2'].cat.codes.head(10)

#
1     13
2     13
3     13
4     13
5     18
6     18
7      7
8      2
9      7
10    18
dtype: int8

用數值表示（0, 1, 2…18）取代原本的標籤值：

In [16]:
pokemon_df['Type 1'] = pokemon_df['Type 1'].cat.codes
pokemon_df['Type 2'] = pokemon_df['Type 2'].cat.codes
pokemon_df.head()  # 顯示頭5比資料

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Bulbasaur,9,13,45,49,49,65,65,45,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,Ivysaur,9,13,60,62,63,80,80,60,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,Venusaur,9,13,80,82,83,100,100,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,Mega Venusaur,9,13,80,100,123,122,120,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,Charmander,6,18,39,52,43,60,50,65,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


將沒有使用到的資料剔除（name）：

In [17]:
pokemon_df.drop('Name', axis='columns', inplace=True)
pokemon_df.head()  # 顯示頭5比資料

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,9,13,45,49,49,65,65,45,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,9,13,60,62,63,80,80,60,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,9,13,80,82,83,100,100,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,9,13,80,100,123,122,120,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,6,18,39,52,43,60,50,65,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


將寶可夢對戰數據中勝利方的表示改為0與1：

In [18]:
# apply方法第一個參數為自訂function,而axis為columns的話會將數據一行一
# 行放入function中處理，最後將所有結果組合成一個數據結構返回。
combats_df['Winner'] = combats_df.apply(lambda x: 0 
                                        if x.Winner == x.First_pokemon else 1, 
                                        axis='columns')
combats_df.head()  # 顯示頭5比資料

,First_pokemon,Second_pokemon,Winner
0,266,298,1
1,702,701,1
2,191,668,1
3,237,683,1
4,151,231,0


### 分割數據集
將數據集分為3個部份分別為：

- 訓練集
- 驗證集
- 測試集

In [19]:
data_num = combats_df.shape[0]
# 取得一筆與data數量相同的亂數索引，主要目的是用於打散資料
indexes = np.random.permutation(data_num)
# 並將亂數索引值分為Train、validation和test分為，這裡劃分比例為6:2:2
train_indexes = indexes[:int(data_num *0.6)]
val_indexes = indexes[int(data_num *0.6):int(data_num *0.8)]
test_indexes = indexes[int(data_num *0.8):]
# 透過上方的索引值從對戰數據中提取資料
train_data = combats_df.loc[train_indexes]
val_data = combats_df.loc[val_indexes]
test_data = combats_df.loc[test_indexes]

### Normalization 標準化

將數值表示的屬性除以19(因為加上empty共有19種屬性)，讓數值縮放置0~1之間。

In [20]:
pokemon_df['Type 1'] = pokemon_df['Type 1'] / 19
pokemon_df['Type 2'] = pokemon_df['Type 2'] / 19


使用Standard Score將生命值、攻擊力和防禦力等數值標準化。

In [21]:
mean = pokemon_df.loc[:, 'HP':'Generation'].mean()  # 計算平均值
std = pokemon_df.loc[:, 'HP':'Generation'].std()  # 計算標準差
# 標準化數據
pokemon_df.loc[:,'HP':'Generation'] = (pokemon_df.loc[:,'HP':'Generation']-mean)/std
pokemon_df.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.473684,0.684211,-0.950032,-0.924328,-0.796655,-0.238981,-0.248033,-0.801002,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,0.473684,0.684211,-0.362595,-0.523803,-0.347700,0.219422,0.290974,-0.284837,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,0.473684,0.684211,0.420654,0.092390,0.293665,0.830626,1.009651,0.403383,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,0.473684,0.684211,0.420654,0.646964,1.576395,1.502951,1.728328,0.403383,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,0.315789,0.947368,-1.185007,-0.831899,-0.989065,-0.391782,-0.787041,-0.112782,-1.398762,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


### 建立Numpy array格式的訓練數據

準備對戰數據中每個寶可夢對應能力值的索引。

In [22]:
x_train_index = np.array(train_data.drop('Winner', axis='columns'))
x_val_index = np.array(val_data.drop('Winner', axis='columns'))
x_test_index = np.array(test_data.drop('Winner', axis='columns'))
print(x_train_index)

[[779 237]
 [158 121]
 [354  59]
 ...
 [275  76]
 [799  82]
 [493 695]]


準備訓練目標

In [23]:
y_train = np.array(train_data['Winner'])
y_val = np.array(val_data['Winner'])
y_test = np.array(test_data['Winner'])

將"One-hot Encode"和"數值"兩種資料分開

In [24]:
# 取得寶可夢的能力值
pokemon_data_normal = np.array(pokemon_df.loc[:, 'HP':'Generation'])
pokemon_data_one_hot = np.array(pokemon_df.loc[:, 'Legendary':])
print(pokemon_data_normal.shape)
print(pokemon_data_one_hot.shape)
# 透過前面準備的索引產生輸入數據
x_train_normal = pokemon_data_normal[x_train_index -1].reshape((-1, 14))
x_val_normal = pokemon_data_normal[x_val_index -1].reshape((-1, 14))
x_test_normal = pokemon_data_normal[x_test_index -1].reshape((-1, 14))
x_train_one_hot = pokemon_data_one_hot[x_train_index -1].reshape((-1, 40))
x_val_one_hot = pokemon_data_one_hot[x_val_index -1].reshape((-1, 40))
x_test_one_hot = pokemon_data_one_hot[x_test_index -1].reshape((-1, 40))
print(x_train_normal.shape)
print(x_train_one_hot.shape)

(800, 7)
(800, 20)
(30000, 14)
(30000, 40)


In [28]:
# 建立normal data輸入層
input_1 = keras.Input(shape=(14, ), name='normal_input')
# 建立 one hot encode輸入層
input_2 = keras.Input(shape=(40, ), name='one_hot_input')

#========================= Write your code here ========================= #
# input_1: 第1層全連接層設為64個unit，relu activation function
x = layers.Dense(64, activation='relu')(input_1)
# input_1: 第2層Dropout層，捨棄比率設定為20%
x = layers.Dropout(0.2) (x)
# input_1: 第3層全連接層設為64個unit，relu activation function
x = layers.Dense(64, activation='relu') (x)

# input_2: 第1層全連接層設為64個unit，relu activation function
x = layers.Dense(64, activation='relu') (input_2)
# input_2: 第2層Dropout層，捨棄比率設定為20%
x = layers.Dropout(0.2) (x)
# input_2: 第3層全連接層設為32個unit，relu activation function
x = layers.Dense(32, activation='relu') (x)

# 第4層Concatenate層，連接input_1和input_2經過全連接後出來的特徵
outputs=layers.Dense(1, activation='relu') (x)
# 第5層Dropout層，捨棄比率設定為20%
x = layers.Dropout(0.2) (x)
# 第6層全連接層設為64個unit，relu activation function
x = layers.Dense(64, activation='relu') (x)
# 第7層Dropout層，捨棄比率設定為20%
x = layers.Dropout(0.2) (x)
# 第8層全連接層設為32個unit，relu activation function
x = layers.Dense(32, activation='relu') (x)
# 第9層Dropout層，捨棄比率設定為20%
x = layers.Dropout(0.2) (x)
# 最後一層全連接層，輸出維度為類別數量，並且使用sigmoid激活函數
outputs=layers.Dense(1, activation='sigmoid') (x)
#======================================================================== #

# 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
model = keras.Model(inputs=[input_1, input_2], outputs=outputs, name='model')
model.summary()  # 顯示網路架構

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
one_hot_input (InputLayer)      [(None, 40)]         0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 64)           2624        one_hot_input[0][0]              
__________________________________________________________________________________________________
dropout_13 (Dropout)            (None, 64)           0           dense_18[0][0]                   
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 32)           2080        dropout_13[0][0]                 
______________________________________________________________________________________________

In [29]:
# 產生網絡拓撲圖
plot_model(model, to_file='model.png')

# 秀出網絡拓撲圖
Image('model.png')

創建紀錄檔目錄

In [30]:
model_dir = 'lab2-logs/models/'
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

建立Callback function：

In [31]:
# 儲存訓練記錄檔
log_dir = os.path.join('lab3-logs', 'model')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
# 儲存最好的網路模型權重
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best-model.h5', 
                                         monitor='val_binary_accuracy', 
                                         save_best_only=True, 
                                         mode='max')

設定訓練使用的優化器、損失函數和指標函數

In [ ]:
model.compile(keras.optimizers.Adam(), 
               loss=keras.losses.BinaryCrossentropy(), 
               metrics=[keras.metrics.BinaryAccuracy()])

訓練網路模型

In [ ]:
history = model.fit(
    x=[x_train_normal, x_train_one_hot], 
    y=y_train, 
    batch_size=64 ,
    epochs=100, 
    validation_data=([x_val_normal, x_val_one_hot], y_val), 
    callbacks=[model_cbk, model_mckp]
)

Epoch 1/100
469/469 [==============================] - 1s 3ms/step - loss: 0.3864 - binary_accuracy: 0.8460 - val_loss: 0.2527 - val_binary_accuracy: 0.9142
Epoch 2/100
469/469 [==============================] - 1s 2ms/step - loss: 0.2609 - binary_accuracy: 0.9056 - val_loss: 0.1900 - val_binary_accuracy: 0.9282
Epoch 3/100
469/469 [==============================] - 1s 2ms/step - loss: 0.2174 - binary_accuracy: 0.9197 - val_loss: 0.1721 - val_binary_accuracy: 0.9354
Epoch 4/100
469/469 [==============================] - 1s 2ms/step - loss: 0.1867 - binary_accuracy: 0.9274 - val_loss: 0.1441 - val_binary_accuracy: 0.9446
Epoch 5/100
469/469 [==============================] - 1s 2ms/step - loss: 0.1674 - binary_accuracy: 0.9370 - val_loss: 0.1310 - val_binary_accuracy: 0.9486
Epoch 6/100
469/469 [==============================] - 1s 2ms/step - loss: 0.1508 - binary_accuracy: 0.9439 - val_loss: 0.1224 - val_binary_accuracy: 0.9505
Epoch 7/100
469/469 [==============================] - 1s 

Epoch 53/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0578 - binary_accuracy: 0.9777 - val_loss: 0.0890 - val_binary_accuracy: 0.9700
Epoch 54/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0606 - binary_accuracy: 0.9772 - val_loss: 0.0887 - val_binary_accuracy: 0.9699
Epoch 55/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0570 - binary_accuracy: 0.9774 - val_loss: 0.0971 - val_binary_accuracy: 0.9652
Epoch 56/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0578 - binary_accuracy: 0.9776 - val_loss: 0.1007 - val_binary_accuracy: 0.9637
Epoch 57/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0609 - binary_accuracy: 0.9765 - val_loss: 0.0902 - val_binary_accuracy: 0.9695
Epoch 58/100
469/469 [==============================] - 1s 2ms/step - loss: 0.0559 - binary_accuracy: 0.9781 - val_loss: 0.0924 - val_binary_accuracy: 0.9676
Epoch 59/100
469/469 [==============================

### 比較兩種網路的訓練結果

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='model-training')
plt.plot(history.history['val_loss'], label='model-validation')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['binary_accuracy'], label='model-training')
plt.plot(history.history['val_binary_accuracy'], label='model-validation')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend()

驗證在測試集上

In [ ]:
# 載入Model準確率最高的模型權重
model.load_weights(model_dir + '/Best-model.h5')
loss, accuracy = model.evaluate([x_test_normal, x_test_one_hot], y_test)
print("Model-1: {}%".format(accuracy))

313/313 [==============================] - 0s 441us/step - loss: 0.0947 - binary_accuracy: 0.9704
Model-1: 0.9703999757766724%
